# Contador de Palavras com PySpark
Este notebook demonstra como usar PySpark para contar palavras em arquivos de texto armazenados no HDFS.

In [1]:
import findspark
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf

## Inicializando Spark

In [2]:
# Parar qualquer SparkContext existente
try:
    spark.stop()
    print("SparkContext anterior finalizado")
except NameError:
    print("Nenhum SparkContext anterior encontrado")
except Exception as e:
    print(f"Erro ao finalizar SparkContext: {e}")

# Inicializando Spark
findspark.init("/usr/spark-3.5.0/")

# Configuração para usar YARN e HDFS
spark = (
    SparkSession.builder.appName("contagem_palavras_lab6")
    .master("yarn")  # Usar YARN agora que está funcionando
    .config("spark.sql.warehouse.dir", "hdfs://spark-master:9000/tmp/hive")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.hadoop.fs.defaultFS", "hdfs://spark-master:9000")
    .getOrCreate()
)

# criar um contexto de sessão do spark (cria um "programa")
sc = spark.sparkContext

print(f"✅ Spark iniciado com sucesso!")
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {sc.master}")
print(f"App Name: {sc.appName}")
print(f"Default FS: {sc._jsc.hadoopConfiguration().get('fs.defaultFS')}")

Nenhum SparkContext anterior encontrado


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/05 14:49:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/05 14:49:10 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


✅ Spark iniciado com sucesso!
Spark Version: 3.5.0
Spark Master: yarn
App Name: contagem_palavras_lab6
Default FS: hdfs://spark-master:9000


## Função Principal - Contagem de Palavras

In [6]:
def main():
    print("Iniciando análise de palavras...")
    
    try:
        # Lendo arquivos do HDFS agora que está funcionando
        print("Lendo arquivos do HDFS...")
        hdfs_path = "hdfs://spark-master:9000/datasets/*.txt"
        
        # Leitura dos arquivos de texto pelo programa spark
        print(f"Lendo arquivos: {hdfs_path}")
        text_files = sc.textFile(hdfs_path)
        
        # Verificar se conseguiu ler algum arquivo
        file_count = text_files.getNumPartitions()
        print(f"Número de partições: {file_count}")
        
        # Transformação: dividir linhas em palavras e limpar
        print("Processando texto...")
        words = (text_files
                .flatMap(lambda line: line.split())
                .filter(lambda word: len(word) > 2)  # Filtrar palavras muito pequenas
                .map(lambda word: ''.join(c.lower() for c in word if c.isalpha()))  # Apenas letras
                .filter(lambda word: len(word) > 0))  # Remover strings vazias
        
        # Contagem de palavras
        print("Contando palavras...")
        word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
        
        # Coletar estatísticas
        total_words = words.count()
        unique_words = word_counts.count()
        
        print(f"\n📊 Estatísticas:")
        print(f"Total de palavras processadas: {total_words:,}")
        print(f"Palavras únicas: {unique_words:,}")
        
        # Mostrar as 15 palavras mais comuns
        print("\n🔝 Top 15 palavras mais comuns:")
        top_words = word_counts.takeOrdered(15, key=lambda x: -x[1])
        for i, (word, count) in enumerate(top_words, 1):
            print(f"  {i:2d}. {word:<15} : {count:>6,}")
        
        # Salvar resultado no HDFS
        output_path = "hdfs://spark-master:9000/datasets_processed/word_count_result"
        print(f"\n💾 Salvando resultado em: {output_path}")
        
        # Remover diretório se existir
        try:
            sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jsc.hadoopConfiguration()).delete(
                sc._jvm.org.apache.hadoop.fs.Path(output_path), True)
        except:
            pass
        
        # Salvar resultado
        word_counts.saveAsTextFile(output_path)
        print("✅ Resultado salvo com sucesso no HDFS!")
        
        # Verificar arquivos salvos
        print("\n📁 Verificando arquivos salvos:")
        saved_files = sc.textFile(output_path + "/part-*")
        print(f"Linhas salvas: {saved_files.count()}")
        
        return True
        
    except Exception as e:
        print(f"❌ Erro durante o processamento: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# Executar a análise
if __name__ == "__main__":
    success = main()
    if success:
        print("\n🎉 Análise concluída com sucesso!")
    else:
        print("\n😞 Análise falhou. Verifique os logs acima.")

Iniciando análise de palavras...
Lendo arquivos do HDFS...
Lendo arquivos: hdfs://spark-master:9000/datasets/*.txt
Número de partições: 5
Processando texto...
Contando palavras...



📊 Estatísticas:
Total de palavras processadas: 614,956
Palavras únicas: 33,976

🔝 Top 15 palavras mais comuns:
   1. the             : 36,279
   2. and             : 21,402
   3. that            : 10,899
   4. was             :  9,166
   5. his             :  8,155
   6. her             :  7,340
   7. you             :  7,174
   8. with            :  6,804
   9. not             :  6,489
  10. had             :  6,323
  11. for             :  6,026
  12. but             :  5,830
  13. she             :  5,726
  14. have            :  4,254
  15. him             :  4,168

💾 Salvando resultado em: hdfs://spark-master:9000/datasets_processed/word_count_result


✅ Resultado salvo com sucesso no HDFS!

📁 Verificando arquivos salvos:
Linhas salvas: 33976

🎉 Análise concluída com sucesso!


## Verificando Resultado Salvo
Vamos verificar o que foi salvo no HDFS:

## Executando a contagem de palavras

In [7]:
# Verificar se o Spark está funcionando antes de executar
if 'spark' in globals() and 'sc' in globals():
    print("Executando análise de palavras...")
    main()
else:
    print("❌ Spark não está inicializado corretamente. Execute a célula de inicialização primeiro.")

Executando análise de palavras...
Iniciando análise de palavras...
Lendo arquivos do HDFS...
Lendo arquivos: hdfs://spark-master:9000/datasets/*.txt
Número de partições: 5
Processando texto...
Contando palavras...



📊 Estatísticas:
Total de palavras processadas: 614,956
Palavras únicas: 33,976

🔝 Top 15 palavras mais comuns:
   1. the             : 36,279
   2. and             : 21,402
   3. that            : 10,899
   4. was             :  9,166
   5. his             :  8,155
   6. her             :  7,340
   7. you             :  7,174
   8. with            :  6,804
   9. not             :  6,489
  10. had             :  6,323
  11. for             :  6,026
  12. but             :  5,830
  13. she             :  5,726
  14. have            :  4,254
  15. him             :  4,168

💾 Salvando resultado em: hdfs://spark-master:9000/datasets_processed/word_count_result


✅ Resultado salvo com sucesso no HDFS!

📁 Verificando arquivos salvos:
Linhas salvas: 33976


## Finalizando Spark

In [8]:
# Finalizar Spark de forma segura
try:
    if 'spark' in globals():
        spark.stop()
        print("✅ Spark finalizado com sucesso!")
    else:
        print("⚠️  Nenhuma sessão Spark ativa encontrada")
except Exception as e:
    print(f"⚠️  Erro ao finalizar Spark: {e}")

✅ Spark finalizado com sucesso!


In [9]:
# Verificar resultado salvo no HDFS
try:
    result_path = "hdfs://spark-master:9000/datasets_processed/word_count_result"
    print(f"📂 Verificando arquivos em: {result_path}")
    
    # Ler resultado salvo
    saved_results = sc.textFile(result_path + "/part-*")
    
    print(f"\n📊 Total de linhas no resultado: {saved_results.count()}")
    print("\n🔍 Primeiras 10 linhas do resultado:")
    
    for i, line in enumerate(saved_results.take(10), 1):
        print(f"  {i:2d}. {line}")
        
except Exception as e:
    print(f"❌ Erro ao verificar resultado: {e}")

📂 Verificando arquivos em: hdfs://spark-master:9000/datasets_processed/word_count_result
❌ Erro ao verificar resultado: 'NoneType' object has no attribute 'sc'


In [10]:
# Finalizar SparkContext (boa prática)
print("\n🔄 Finalizando Spark...")
try:
    spark.stop()
    print("✅ SparkContext finalizado com sucesso!")
except Exception as e:
    print(f"⚠️ Erro ao finalizar: {e}")


🔄 Finalizando Spark...
✅ SparkContext finalizado com sucesso!
